<a href="https://colab.research.google.com/github/praffuln/generative-ai/blob/master/rag/rag-1/rag_routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello world')

hello world


**Enviornment**

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube google-generativeai langchain_google_genai langchain_community


# Setup Keys For GOOGLE_API_KEY AND LANGSMITH

In [7]:
# Assigning value to variable
GOOGLE_API_KEY=''
LANGCHAIN_API_KEY = ''


# Imports os, google Gemini EMbedding and Chat Classes with FAISS vectorstores

In [8]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS



# setup langsmith for tracing

In [9]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## [Logical and Semantic routing]

Use function-calling for classification.



In [34]:
# from typing import Literal

# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import BaseModel, Field
# from langchain_openai import ChatOpenAI

# # Data model
# class RouteQuery(BaseModel):
#     """Route a user query to the most relevant datasource."""

#     datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
#         ...,
#         description="Given a user question choose which datasource would be most relevant for answering their question",
#     )

# # LLM with function call
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3) # Try using gemini-pro-vision
# structured_llm = llm.with_structured_output(RouteQuery)



# # Prompt
# # Prompt
# system = """You are an expert at routing a user question to the appropriate data source.

# Based on the programming language the question is referring to, route it to the relevant data source."""


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "{question}"),
#     ]
# )

# # Define router
# router = prompt | structured_llm





from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3) # Try using gemini-pro-vision
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source by selecting one of the following options:

python_docs
js_docs
golang_docs
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router
router = prompt | structured_llm
print(router)


first=ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at routing a user question to the appropriate data source.\n\nBased on the programming language the question is referring to, route it to the relevant data source by selecting one of the following options:\n\npython_docs\njs_docs\ngolang_docs \n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]) middle=[RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', temperature=0.3, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x77ff35042950>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x77ff3506fc40>, default_metadata=()), kwargs={'tools': [{'function_declarations': [{'name': 'RouteQuery', 'description

In [35]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})
print(result)


datasource='python_docs'


In [36]:
result

RouteQuery(datasource='python_docs')

In [37]:
result.datasource


'python_docs'

In [38]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here
        return "chain for js_docs"
    else:
        ### Logic here
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)


In [39]:
full_chain.invoke({"question": question})


'chain for python_docs'